In [ ]:
%%capture
!pip install openai

In [1]:
import numpy as np
# import openai
import os
import pandas as pd
from   pathlib import Path
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


In [2]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Mounted at /content/drive


In [3]:
interp_dir = '/content/drive/MyDrive/legal_interpretation/code/generative_testing'
train_test_dir = os.path.join(interp_dir, 'train_test_splits', 'train_test_splits_2')
output_path = os.path.join(interp_dir, 'few_shot_results')
generated_output_path = os.path.join(output_path, 'generations')
descriptive_errors_dir = os.path.join(output_path, 'errors')

In [4]:
worksheet = gc.open('final_cleaned_paragraphs').sheet1
rows = worksheet.get_all_values()
interpretation_df = pd.DataFrame(rows)

In [5]:
interpretation_df.columns = interpretation_df.iloc[0]
interpretation_df = interpretation_df.iloc[1:]

In [ ]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


device_name = 'cuda'
max_length = 512

In [ ]:
openai.api_key = open(os.path.join(interp_dir, 'private', 'openai_key.txt')).read().strip()

In [ ]:
prompt_text = "Determine the legal interpretation used in the following passage. Return a single choice from GRAND, FORMAL, or NONE. Here are examples:"
formal_example = "Accepting this point, too, for argument's sake, the question becomes: What did \"discriminate\" mean in 1964? As it turns out, it meant then roughly what it means today: \"To make a difference in treatment or favor (of one as compared with others).\" Webster's New International Dictionary 745 (2d ed. 1954). To \"discriminate against\" a person, then, would seem to mean treating that individual worse than others who are similarly situated. [CITE]. In so-called \"disparate treatment\" cases like today's, this Court has also held that the difference in treatment based on sex must be intentional. See, e.g., [CITE]. So, taken together, an employer who intentionally treats a person worse because of sex—such as by firing the person for actions or attributes it would tolerate in an individual of another sex—discriminates against that person in violation of Title VII."
grand_example = "Respondent's argument is not without force. But it overlooks the significance of the fact that the Kaiser-USWA plan is an affirmative action plan voluntarily adopted by private parties to eliminate traditional patterns of racial segregation. In this context respondent's reliance upon a literal construction of §§ 703 (a) and (d) and upon McDonald is misplaced. See [CITE]. It is a \"familiar rule, that a thing may be within the letter of the statute and yet not within the statute, because not within its spirit, nor within the intention of its makers.\" [CITE]. The prohibition against racial discrimination in §§ 703 (a) and (d) of Title VII must therefore be read against the background of the legislative history of Title VII and the historical context from which the Act arose. See [CITE]. Examination of those sources makes clear that an interpretation of the sections that forbade all race-conscious affirmative action would \"bring about an end completely at variance with the purpose of the statute\" and must be rejected. [CITE]. See [CITE]."
none_example = "The questions are, What is the form of an assignment, and how must it be evidenced? There is no precise form. It may be. by delivery. Briggs v. Dorr, CITE, citing numerous cases; Onion v. Paul, 1 Har. & Johns. 114; Dunn v. Snell, CITE; Titcomb v. Thomas, 5 Greenl. 282. True, it is said it must be on a valuable consideration, with intent to transfer it. But these last are requisites in all assignments, or transfers of securities, negotiable or not. It may be by writing under seal, by writing without seal, by oral declarations, accompanied in all cases by delivery, and on a just consideration. The evidence may be by proof of handwriting and proof of. possession. It may be proved by proving the signature of the payee or obligee on the back, and possession by a third person. 3 Gill & Johns. 218"

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def get_few_shot_interp(text, formal_example, grand_example, none_example):
    prompt = f"{prompt_text}\n\n###\nText: {formal_example}\nFORMAL\n###\nText: {grand_example}\nGRAND\n###\nText: {none_example}\nNONE\n###\nText: {text}\n"
    response = openai.ChatCompletion.create(
        model='gpt-4',
        max_tokens=4,
        messages = [{'role': 'user', 'content': prompt + text}]
    )
    return response['choices'][0]['message']['content'].strip().lower()

In [6]:
interpretation_df = interpretation_df[interpretation_df['class'].notna()]

In [ ]:
interpretation_df.sample(5)["paragraph"].tolist()[2]

'It is only recently that we gave careful consideration to the problems of review of Tax Court decisions. Dobson v. Commissioner, 320 U. S. 489. That opinion emphasized that our interpretation of Congressional purpose, in enacting the statute, just quoted, for judicial review of Tax Court decisions, was that Congress intended to leave to the final determination of the Tax Court all issues which were not clear-cut questions of law.'

In [7]:
macro_f1_l = []
macro_precision_l = []
macro_recall_l = []

weighted_f1_l = []
weighted_precision_l = []
weighted_recall_l = []

grand_f1_l = []
grand_precision_l = []
grand_recall_l = []

formal_f1_l = []
formal_precision_l = []
formal_recall_l = []

none_f1_l = []
none_precision_l = []
none_recall_l = []

In [8]:
full_df = pd.DataFrame()

In [ ]:
for split in range(2, 5):

  split_id_file = os.path.join(train_test_dir, f'split_{split}')

  with open(split_id_file, 'r') as file:
      train_ids = file.read().split("\n")

  interpretation_train_df = interpretation_df[interpretation_df["section_id"].isin(train_ids)]
  interpretation_test_df = interpretation_df[~interpretation_df["section_id"].isin(train_ids)]


  X_test = interpretation_test_df["paragraph"].to_list()
  y_test = interpretation_test_df["class"].to_list()


  total = len(X_test)

  predicted_labels = []
  for i, text in enumerate(X_test):
    prediction = get_few_shot_interp(text, formal_example, grand_example, none_example).upper()
    predicted_labels.append(prediction)

    if i % 50 == 0:
      precent = round((i/total)*100, 2)
      print(f"{precent}% through processing.")

  with open(os.path.join(generated_output_path, f'predictions_{split}.txt'), 'w') as file:
     for label in predicted_labels:
        file.write(f"{label}\n")

  predictions_df = pd.DataFrame(
  {'section_id': interpretation_test_df["section_id"].tolist(),
    'gold': y_test,
    'predicted': predicted_labels,
    'text': X_test
  })
  errors_df = predictions_df.query('gold != predicted')
  errors_df.to_csv(os.path.join(descriptive_errors_dir, f"{split}_errors.csv"))

  # class_report = classification_report(y_test, predicted_labels, output_dict=True)

  # sample_dict = {
  #     "model": "descriptive_generative",
  #     "split": split,

  #     "macro_f1": round(class_report["macro avg"]["f1-score"], 3),
  #     "macro_precision": round(class_report["macro avg"]["precision"], 3),
  #     "macro_recall": round(class_report["macro avg"]["recall"], 3),

  #     "weighted_f1": round(class_report["weighted avg"]["f1-score"], 3),
  #     "weighted_precision": round(class_report["weighted avg"]["precision"], 3),
  #     "weighted_recall": round(class_report["weighted avg"]["recall"], 3),

  #     "grand_f1": round(class_report["GRAND"]["f1-score"], 3),
  #     "grand_precision": round(class_report["GRAND"]["precision"], 3),
  #     "grand_recall": round(class_report["GRAND"]["recall"], 3),

  #     "formal_f1": round(class_report["FORMAL"]["f1-score"], 3),
  #     "formal_precision": round(class_report["FORMAL"]["precision"], 3),
  #     "formal_recall": round(class_report["FORMAL"]["recall"], 3),

  #     "none_f1": round(class_report["NONE"]["f1-score"], 3),
  #     "none_precision": round(class_report["NONE"]["precision"], 3),
  #     "none_recall": round(class_report["NONE"]["recall"], 3),
  # }

  # new_row = pd.DataFrame(sample_dict, index = [0])
  # full_df = pd.concat([full_df, new_row])


  # macro_f1_l.append(class_report["macro avg"]["f1-score"])
  # macro_precision_l.append(class_report["macro avg"]["precision"])
  # macro_recall_l.append(class_report["macro avg"]["recall"])

  # weighted_f1_l.append(class_report["weighted avg"]["f1-score"])
  # weighted_precision_l.append(class_report["weighted avg"]["precision"])
  # weighted_recall_l.append(class_report["weighted avg"]["recall"])

  # grand_f1_l.append(class_report["GRAND"]["f1-score"])
  # grand_precision_l.append(class_report["GRAND"]["precision"])
  # grand_recall_l.append(class_report["GRAND"]["recall"])

  # formal_f1_l.append(class_report["FORMAL"]["f1-score"])
  # formal_precision_l.append(class_report["FORMAL"]["precision"])
  # formal_recall_l.append(class_report["FORMAL"]["recall"])

  # none_f1_l.append(class_report["NONE"]["f1-score"])
  # none_precision_l.append(class_report["NONE"]["precision"])
  # none_recall_l.append(class_report["NONE"]["recall"])

0.0% through processing.
7.28% through processing.
14.56% through processing.
21.83% through processing.
29.11% through processing.
36.39% through processing.
43.67% through processing.
50.95% through processing.
58.22% through processing.
65.5% through processing.
72.78% through processing.
80.06% through processing.
87.34% through processing.
94.61% through processing.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0% through processing.
7.28% through processing.
14.56% through processing.
21.83% through processing.
29.11% through processing.
36.39% through processing.
43.67% through processing.
50.95% through processing.
58.22% through processing.
65.5% through processing.
72.78% through processing.
80.06% through processing.
87.34% through processing.
94.61% through processing.
0.0% through processing.
7.28% through processing.
14.56% through processing.
21.83% through processing.
29.11% through processing.
36.39% through processing.
43.67% through processing.
50.95% through processing.
58.22% through processing.
65.5% through processing.
72.78% through processing.
80.06% through processing.
87.34% through processing.
94.61% through processing.
0.0% through processing.
7.28% through processing.
14.56% through processing.
21.83% through processing.
29.11% through processing.
36.39% through processing.
43.67% through processing.
50.95% through processing.
58.22% through processing.
65.5% throug

In [9]:
for split in range(0, 5):

  split_id_file = os.path.join(train_test_dir, f'split_{split}')

  with open(split_id_file, 'r') as file:
      train_ids = file.read().split("\n")

  interpretation_train_df = interpretation_df[interpretation_df["section_id"].isin(train_ids)]
  interpretation_test_df = interpretation_df[~interpretation_df["section_id"].isin(train_ids)]


  X_test = interpretation_test_df["paragraph"].to_list()
  y_test = interpretation_test_df["class"].to_list()

  with open(os.path.join(generated_output_path, f'predictions_{split}.txt'), 'r') as file:
    print(file)
    predicted_labels = [line.rstrip() for line in file]

  print(y_test, predicted_labels)
  class_report = classification_report(y_test, predicted_labels, output_dict=True)

  sample_dict = {
      "model": "class_examples",
      "split": split,

      "macro_f1": round(class_report["macro avg"]["f1-score"], 3),
      "macro_precision": round(class_report["macro avg"]["precision"], 3),
      "macro_recall": round(class_report["macro avg"]["recall"], 3),

      "weighted_f1": round(class_report["weighted avg"]["f1-score"], 3),
      "weighted_precision": round(class_report["weighted avg"]["precision"], 3),
      "weighted_recall": round(class_report["weighted avg"]["recall"], 3),

      "grand_f1": round(class_report["GRAND"]["f1-score"], 3),
      "grand_precision": round(class_report["GRAND"]["precision"], 3),
      "grand_recall": round(class_report["GRAND"]["recall"], 3),

      "formal_f1": round(class_report["FORMAL"]["f1-score"], 3),
      "formal_precision": round(class_report["FORMAL"]["precision"], 3),
      "formal_recall": round(class_report["FORMAL"]["recall"], 3),

      "none_f1": round(class_report["NONE"]["f1-score"], 3),
      "none_precision": round(class_report["NONE"]["precision"], 3),
      "none_recall": round(class_report["NONE"]["recall"], 3),

  }

  new_row = pd.DataFrame(sample_dict, index = [0])
  full_df = pd.concat([full_df, new_row])

  macro_f1_l.append(class_report["macro avg"]["f1-score"])
  macro_precision_l.append(class_report["macro avg"]["precision"])
  macro_recall_l.append(class_report["macro avg"]["recall"])

  weighted_f1_l.append(class_report["weighted avg"]["f1-score"])
  weighted_precision_l.append(class_report["weighted avg"]["precision"])
  weighted_recall_l.append(class_report["weighted avg"]["recall"])

  grand_f1_l.append(class_report["GRAND"]["f1-score"])
  grand_precision_l.append(class_report["GRAND"]["precision"])
  grand_recall_l.append(class_report["GRAND"]["recall"])

  formal_f1_l.append(class_report["FORMAL"]["f1-score"])
  formal_precision_l.append(class_report["FORMAL"]["precision"])
  formal_recall_l.append(class_report["FORMAL"]["recall"])

  none_f1_l.append(class_report["NONE"]["f1-score"])
  none_precision_l.append(class_report["NONE"]["precision"])
  none_recall_l.append(class_report["NONE"]["recall"])


<_io.TextIOWrapper name='/content/drive/MyDrive/legal_interpretation/code/generative_testing/few_shot_results/generations/predictions_0.txt' mode='r' encoding='UTF-8'>
['NONE', 'GRAND', 'NONE', 'GRAND', 'FORMAL', 'FORMAL', 'GRAND', 'NONE', 'NONE', 'GRAND', 'FORMAL', 'GRAND', 'GRAND', 'GRAND', 'FORMAL', 'FORMAL', 'NONE', 'GRAND', 'FORMAL', 'NONE', 'NONE', 'NONE', 'NONE', 'GRAND', 'FORMAL', 'FORMAL', 'GRAND', 'FORMAL', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'GRAND', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'GRAND', 'FORMAL', 'FORMAL', 'NONE', 'GRAND', 'NONE', 'GRAND', 'NONE', 'GRAND', 'NONE', 'NONE', 'NONE', 'GRAND', 'FORMAL', 'NONE', 'NONE', 'GRAND', 'GRAND', 'NONE', 'NONE', 'NONE', 'GRAND', 'GRAND', 'GRAND', 'NONE', 'GRAND', 'GRAND', 'NONE', 'FORMAL', 'NONE', 'GRAND', 'NONE', 'NONE', 'NONE', 'NONE', 'GRAND', 'NONE', 'NONE', 'FORMAL', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'GRAND', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', '

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<_io.TextIOWrapper name='/content/drive/MyDrive/legal_interpretation/code/generative_testing/few_shot_results/generations/predictions_2.txt' mode='r' encoding='UTF-8'>
['FORMAL', 'NONE', 'FORMAL', 'GRAND', 'GRAND', 'GRAND', 'GRAND', 'GRAND', 'FORMAL', 'NONE', 'FORMAL', 'GRAND', 'FORMAL', 'FORMAL', 'GRAND', 'NONE', 'NONE', 'FORMAL', 'NONE', 'GRAND', 'GRAND', 'FORMAL', 'NONE', 'NONE', 'NONE', 'GRAND', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'GRAND', 'GRAND', 'GRAND', 'FORMAL', 'NONE', 'NONE', 'NONE', 'GRAND', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'FORMAL', 'GRAND', 'FORMAL', 'NONE', 'NONE', 'GRAND', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'GRAND', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'FORMAL', 'NONE', 'NONE', 'GRAND', 'NONE', 'NONE', 'NONE', 'NONE', 'NONE', 'GRAND', 'GRAND', 'NONE', 'NONE', 'GRAND', 'NONE', 'NONE', 'N

In [10]:
macro_f1 = sum(macro_f1_l) / len(macro_f1_l)
macro_precision = sum(macro_precision_l) / len(macro_precision_l)
macro_recall = sum(macro_recall_l) / len(macro_recall_l)

weighted_f1 = sum(weighted_f1_l) / len(weighted_f1_l)
weighted_precision = sum(weighted_precision_l) / len(weighted_precision_l)
weighted_recall = sum(weighted_recall_l) / len(weighted_recall_l)

grand_f1 = sum(grand_f1_l) / len(grand_f1_l)
grand_precision = sum(grand_precision_l) / len(grand_precision_l)
grand_recall = sum(grand_recall_l) / len(grand_recall_l)

formal_f1 = sum(formal_f1_l) / len(formal_f1_l)
formal_precision = sum(formal_precision_l) / len(formal_precision_l)
formal_recall = sum(formal_recall_l) / len(formal_recall_l)

none_f1 = sum(none_f1_l) / len(none_f1_l)
none_precision = sum(none_precision_l) / len(none_precision_l)
none_recall = sum(none_recall_l) / len(none_recall_l)



In [11]:

model_dict = {
    "model": "examples_generative",
    "split": "averages",

    "macro_f1": round(macro_f1, 3),
    "macro_precision": round(macro_precision, 3),
    "macro_recall": round(macro_recall, 3),

    "weighted_f1": round(weighted_f1, 3),
    "weighted_precision": round(weighted_precision, 3),
    "weighted_recall": round(weighted_recall, 3),

    "grand_f1": round(grand_f1, 3),
    "grand_precision": round(grand_precision, 3),
    "grand_recall": round(grand_recall, 3),

    "formal_f1": round(formal_f1, 3),
    "formal_precision": round(formal_precision, 3),
    "formal_recall": round(formal_recall, 3),

    "none_f1": round(none_f1, 3),
    "none_precision": round(none_precision, 3),
    "none_recall": round(none_recall, 3),
}

new_row = pd.DataFrame(model_dict, index = [0])
full_df = pd.concat([full_df, new_row])

full_df.to_csv(os.path.join(output_path, 'gpt_generated_examples_results.csv'))



In [12]:
full_df

,model,split,macro_f1,macro_precision,macro_recall,weighted_f1,weighted_precision,weighted_recall,grand_f1,grand_precision,grand_recall,formal_f1,formal_precision,formal_recall,none_f1,none_precision,none_recall
0,class_examples,0,0.477,0.497,0.577,0.546,0.692,0.515,0.458,0.370,0.600,0.371,0.257,0.667,0.603,0.864,0.464
0,class_examples,1,0.356,0.371,0.416,0.559,0.687,0.523,0.470,0.405,0.561,0.327,0.223,0.608,0.628,0.857,0.496
0,class_examples,2,0.500,0.515,0.613,0.571,0.717,0.540,0.477,0.384,0.629,0.399,0.275,0.725,0.626,0.885,0.484
0,class_examples,3,0.474,0.496,0.561,0.552,0.694,0.517,0.446,0.365,0.571,0.356,0.248,0.631,0.619,0.874,0.479
0,class_examples,4,0.462,0.477,0.541,0.534,0.650,0.504,0.416,0.350,0.513,0.367,0.259,0.634,0.603,0.821,0.477
0,examples_generative,averages,0.454,0.471,0.541,0.552,0.688,0.520,0.453,0.375,0.575,0.364,0.252,0.653,0.616,0.860,0.480


In [13]:
averages_df = full_df[full_df["split"] == "averages"]

In [14]:
averages_df = averages_df.drop(columns = ["weighted_f1", "weighted_precision", "weighted_recall"])

In [15]:
print(averages_df.to_latex(
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
                  index = False
                  ))

\begin{tabular}{llrrrrrrrrrrrr}
\toprule
              model &    split &  macro\_f1 &  macro\_precision &  macro\_recall &  grand\_f1 &  grand\_precision &  grand\_recall &  formal\_f1 &  formal\_precision &  formal\_recall &  none\_f1 &  none\_precision &  none\_recall \\
\midrule
examples\_generative & averages &      0.45 &             0.47 &          0.54 &      0.45 &             0.38 &          0.57 &       0.36 &              0.25 &           0.65 &     0.62 &            0.86 &         0.48 \\
\bottomrule
\end{tabular}



<ipython-input-15-7328dfe888dc>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(averages_df.to_latex(
